In [2]:
import pandas as pd
from datetime import datetime, timedelta

In [3]:
def texto_a_fecha(texto):
    hoy = datetime.now()
    palabras = texto.split()
    
    if len(palabras) < 2:
        return hoy.strftime('%m/%d/%Y')
    
    cantidad = palabras[1]
    unidad = palabras[2]
    
    if cantidad == 'un' or cantidad == 'una':
        cantidad = 1
    else:
        try:
            cantidad = int(cantidad)
        except ValueError:
            return hoy.strftime('%m/%d/%Y')
    
    if 'año' in unidad:
        return (hoy - timedelta(days=cantidad*365)).strftime('%m/%d/%Y')
    elif 'mes' in unidad:
        return (hoy - timedelta(days=cantidad*30)).strftime('%m/%d/%Y')
    elif 'semana' in unidad:
        return (hoy - timedelta(weeks=cantidad)).strftime('%m/%d/%Y')
    else:
        return hoy.strftime('%m/%d/%Y')

In [5]:
name = 'bares'
df = pd.read_csv(f'comentarios_{name}.xls')
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
df['Fecha_numero'] = df['Fecha'].apply(texto_a_fecha)
df.to_csv(f'comentarios_{name}.csv', index=False)

In [6]:
list(df['Nombre'].unique())

['SINNERS Microcervecería Floresta',
 'La Santa Cantina Pop',
 'Holy Krank Pub',
 'Abysmo GastroPub UIO',
 'Zmus Bar',
 "Finn McCool's",
 nan,
 'Indie Beer Company',
 'Lucky Charm Pub & Beer',
 'Rock & Beef Resto-Bar',
 'Odeon Pub House',
 'The Boot Sports Bar',
 'Duff Resto Bar',
 'Tijuana']

# Mapa interactivo

In [2]:
import folium
from folium.plugins import MiniMap

In [57]:
def mapa_floresta(coord = [-0.2086949, -78.4854354], nombre= 'Pollos Asados'):
    popuptext = f'<b>{nombre}</b>'
    floresta = folium.Map(location = coord, zoom_start=18)
    folium.Marker(location= coord, popup= popuptext ).add_to(floresta)
    folium.Circle(location= coord, color = 'purple', fill_color = 'red', radius = 10, weight = 4, fill_opacity = 0.5, tooltip = f'{nombre}').add_to(floresta)
    minimap = MiniMap()
    floresta.add_child(minimap)
    return floresta

In [58]:
df = pd.read_csv(f'comentarios_bares.csv')

In [59]:
def extraer_coordenada(df, nombre):
    df = df.pivot_table(index='Nombre', values=['Latitud', 'Longitud'], aggfunc='first')
    df = df.loc[f'{nombre}',['Latitud', 'Longitud']]
    return [df['Latitud'], df['Longitud']]
coord = extraer_coordenada(df, 'Abysmo GastroPub UIO')
mapa_floresta(coord, 'Abysmo GastroPub UIO')